In [328]:
print("hello world")

hello world


In [329]:
# get some libraries that will be useful
import re
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.image as mpimg
import stop_words
from sklearn.ensemble import AdaBoostClassifier

# the Naive Bayes model
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
# function to split the data for cross-validation
from sklearn.model_selection import train_test_split
# function for transforming documents into counts
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# function for encoding categories
from sklearn.preprocessing import LabelEncoder

# Outputs the headlines dataset.
# X is a list of headlines
# y is a list of binary labels, 0 stands for Haaretz and 1 for Israel Hayom
def load_dataset(filenames=['haaretz.csv','israelhayom.csv']):
    cur_y = 0
    X = pd.DataFrame()
    y = np.empty(0,dtype=np.int32)
    for filename in filenames:
        train_cur = pd.read_csv(filename, header=None)
        X = pd.concat([X,train_cur[0]])
        y = np.append(y,cur_y*np.ones(len(train_cur),dtype=np.int32))
        cur_y += 1
    X = [x[0] for x in X.values.tolist()]
    y = y.tolist()
    return X,y

# grab the data
X,Y = load_dataset()

In [330]:
def normalize_text(s):
    s = s.lower()
    
    # remove punctuation that is not word-internal (e.g., hyphens, apostrophes)
    #s = re.sub('\s\W',' ',s)
   # s = re.sub('\W\s',' ',s)
    
    # make sure we didn't introduce any double spaces
    s = re.sub('\s+',' ',s)
    
    return s

In [331]:
X = [normalize_text(s) for s in X]

In [346]:
stop_words_list=stop_words.get_stop_words('english')
##stop_words_list=[]


# take a look at the shape of each of these


In [347]:
#svm 
# pull the data into vectors
#stop_words_list=stop_words.get_stop_words('english')
#stop_words_list=[]
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1,6), analyzer='char_wb', lowercase=False, strip_accents='unicode', stop_words=stop_words_list)
x = vectorizer.fit_transform(X)
encoder = LabelEncoder()
y = encoder.fit_transform(Y)

# split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
#print("y %", (y_train > 0).sum()/(y_train.shape))

svm1 = svm.LinearSVC()
svm1.fit(x_train, y_train)
print("svm", svm1.score(x_test, y_test))

svm 0.853658536585


In [334]:
#nb
# pull the data into vectors
vectorizer = CountVectorizer(stop_words=stop_words_list)
x = vectorizer.fit_transform(X)
encoder = LabelEncoder()
y = encoder.fit_transform(Y)

# split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
#print("y %", (y_train > 0).sum()/(y_train.shape))

nb = MultinomialNB()
nb.fit(x_train, y_train)
print("naive baise", nb.score(x_test, y_test))


#adaboost
ab = AdaBoostClassifier()
#ab.estimators_=[svm.LinearSVC()]
ab.fit(x_train,y_train)
print("adaboost", ab.score(x_test, y_test))

naive baise 0.749237804878
adaboost 0.723323170732


In [336]:
#from sklearn.calibration import CalibratedClassifierCV
#from sklearn.model_selection import LeaveOneOut

#num_classes = 2
#X, y = datasets.make_classification(n_samples=100, n_features=20,
#                                    n_informative=18, n_redundant=2,
#                    n_classes=num_classes)
#clf = LinearSVC()
#clf_prob = CalibratedClassifierCV(clf, method="sigmoid", cv=LeaveOneOut())
#clf_prob.fit(X, y)


In [337]:
try1 = ["Occupation Occupation Occupation", "west bank", "right wing","sarah","Bibi and Yair",
                             "Israeli IDF iran minister"]

test = vectorizer.transform(try1)
res= nb.predict(test)

for i,cur in enumerate(try1):
    tag = "haaretz" if res[i] == 0 else "Israel Hayom"
    print(cur, ": ", tag)


Occupation Occupation Occupation :  haaretz
west bank :  haaretz
right wing :  haaretz
sarah :  haaretz
Bibi and Yair :  haaretz
Israeli IDF iran minister :  Israel Hayom


In [338]:
z = [1,2,3,4,5,6]
from sklearn.model_selection import KFold
kf = KFold(n_splits = 2,shuffle=True)
kf.get_n_splits(z)
print([a for a in kf.split(z)])

[(array([0, 2, 5]), array([1, 3, 4])), (array([1, 3, 4]), array([0, 2, 5]))]
